In [1]:
import requests

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
r19 = requests.get("https://dprc3-123.wrap.gov.tw/FuncModule/RiverSea/WebService/RiverSea.ashx?cmd=hyinfo&sn=201103&spid=02&name=%E6%96%B0%E7%AB%B9%E6%BC%81%E6%B8%AF%E5%A4%96%E6%B5%B7CA01&class=current&st=2011/03/19&et=2011/04/15&_=1592981418270", verify=False)

# 輸入url

print(r19.status_code)
type(r19)
#測試看有沒有讀取成功：
#向網路伺服器發出請求時都會回傳 HTTP 狀態碼，表示有關請求的情況，常見的有：
#200 一切順利，結果已經回傳
#301 伺服器將使用者重新定向（re-direct）到另一個位址，當網站更換網域名稱或更改 Routes 時可能會發生
#400 錯誤的請求
#401 未通過伺服器的身份驗證，當請求沒有一併發送正確憑證時會發生。
#403 伺服器已經理解請求，但是拒絕執行它，意即與請求一併發送的憑證無效
#404 請求失敗

D:\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200


requests.models.Response

In [4]:
list_of_dicts_19 = r19.json()

In [6]:
def get_target_value(key, dic, tmp_list):
    """
    :param key: 目标key值
    :param dic: JSON数据
    :param tmp_list: 用于存储获取的数据
    :return: list
    """
    if not isinstance(dic, dict) or not isinstance(tmp_list, list):  # 对传入数据进行格式校验
        return 'argv[1] not an dict or argv[-1] not an list '

    if key in dic.keys():
        tmp_list.append(dic[key])  # 传入数据存在则存入tmp_list

    for value in dic.values():  # 传入数据不符合则对其value值进行遍历
        if isinstance(value, dict):
            get_target_value(key, value, tmp_list)  # 传入数据的value值是字典，则直接调用自身
        elif isinstance(value, (list, tuple)):
            _get_value(key, value, tmp_list)  # 传入数据的value值是列表或者元组，则调用_get_value


    return tmp_list


def _get_value(key, val, tmp_list):
    for val_ in val:
        if isinstance(val_, dict):  
            get_target_value(key, val_, tmp_list)  # 传入数据的value值是字典，则调用get_target_value
        elif isinstance(val_, (list, tuple)):
            _get_value(key, val_, tmp_list)   # 传入数据的value值是列表或者元组，则调用自身
            
            
#主要核心code，多層dict擷取資料，直接套用即可

In [7]:
vew_list = get_target_value('VEW', list_of_dicts_19, [])
vns_list = get_target_value('VNS', list_of_dicts_19, [])
vv_list = get_target_value('VV', list_of_dicts_19, [])
cd_list = get_target_value('CD', list_of_dicts_19, [])
time_list = get_target_value('DataTime', list_of_dicts_19, [])

In [8]:
combine_data = {
                'DataTime': time_list,
                'VEW(東西向流速)': vew_list,
                'VNS(南北向流速)': vns_list,
                'VV(總流速)': vv_list,
                'CD(總流向)': cd_list,
               }

# 將讀取的各項list配合其column名稱轉成dict, 以方便做成DataFrame

In [9]:
df = pd.DataFrame(combine_data)

In [10]:
df.head()

,DataTime,VEW(東西向流速),VNS(南北向流速),VV(總流速),CD(總流向)
0,2011/03/19 14:00:00,28,57,1,26
1,2011/03/19 14:00:00,32,62,2,27.8999996185303
2,2011/03/19 14:00:00,37,64,2,29.8999996185303
3,2011/03/19 14:00:00,40,68,2,30.7999992370605
4,2011/03/19 14:00:00,45,70,2,32.5999984741211


In [11]:
writer=pd.ExcelWriter('shortterm_obs_test.xlsx')
df.to_excel(writer, sheet_name = '新竹漁港外海CA0')

writer.save()

#將各別df存進同一excel中之不同sheet